In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
PolRel = pd.read_excel("Seshat_PolityRelationData.xlsx")
Data = pd.read_csv("Seshat Data.csv")

In [3]:
set(Data["NGA"]).issubset(set(PolRel["NGA"]))

True

In [4]:
# Discard superfluous NGAs
SubsetNGA = np.array(PolRel["NGA"].isin(set(Data["NGA"])))
PolRel = PolRel.iloc[SubsetNGA]


In [5]:
PolRel.head(50)

,NGA,PolID,Longform.Name,Polity.Start,Polity.End,Dupl,Section,Variable,Value.From,Value.To,Value.Date.From,Value.Date.To
13,Big Island Hawaii,Hawaii1,Hawaii I,1000.0,1200.0,n,General variables,succeeding (quasi)polity,Hawaii II,NaN,NaN,NaN
14,Big Island Hawaii,Hawaii2,Hawaii II,1200.0,1580.0,n,General variables,succeeding (quasi)polity,Hawaii III,NaN,NaN,NaN
15,Big Island Hawaii,Hawaii2,Hawaii II,1200.0,1580.0,n,General variables,relationship to preceding (quasi)polity,continuity,NaN,NaN,NaN
16,Big Island Hawaii,Hawaii3,Hawaii III,1580.0,1778.0,n,General variables,succeeding (quasi)polity,Kingdom of Hawaii - Kamehameha Period,NaN,NaN,NaN
17,Cahokia,USMisEW,Cahokia - Early Woodland,-600.0,-150.0,n,General variables,succeeding (quasi)polity,Cahokia - Middle Woodland,NaN,NaN,NaN
18,Cahokia,USMisEW,Cahokia - Early Woodland,-600.0,-150.0,n,General variables,relationship to preceding (quasi)polity,continuity,NaN,NaN,NaN
19,Cahokia,USMisMW,Cahokia - Middle Woodland,-150.0,300.0,n,General variables,succeeding (quasi)polity,Cahokia - Late Woodland I,NaN,NaN,NaN
20,Cahokia,USMisMW,Cahokia - Middle Woodland,-150.0,300.0,n,General variables,relationship to preceding (quasi)polity,Continuity,NaN,NaN,NaN
21,Cahokia,USMisRo,Cahokia - Late Woodland I,300.0,450.0,n,General variables,succeeding (quasi)polity,Cahokia - Late Woodland II,NaN,NaN,NaN
22,Cahokia,USMisRo,Cahokia - Late Woodland I,300.0,450.0,n,General variables,relationship to preceding (quasi)polity,Continuity,NaN,NaN,NaN


In [6]:
for i in list(set(PolRel["Dupl"])):
    print(i,sum(PolRel["Dupl"] == i))

n 596
y 103


In [19]:
# relationship to preceding (quasi)polity

for v in list(set(PolRel["Variable"])):
    print(v, sum(PolRel["Variable"]==v))

# Logical for Variable is the relationship
VR = PolRel["Variable"] == "relationship to preceding (quasi)polity"

Previous = PolRel.loc[np.logical_not(VR)]
Relationships = PolRel.loc[VR]

relationship to preceding (quasi)polity 335
succeeding (quasi)polity 364


In [23]:
Previous.head()

,NGA,PolID,Longform.Name,Polity.Start,Polity.End,Dupl,Section,Variable,Value.From,Value.To,Value.Date.From,Value.Date.To
13,Big Island Hawaii,Hawaii1,Hawaii I,1000.0,1200.0,n,General variables,succeeding (quasi)polity,Hawaii II,NaN,NaN,NaN
14,Big Island Hawaii,Hawaii2,Hawaii II,1200.0,1580.0,n,General variables,succeeding (quasi)polity,Hawaii III,NaN,NaN,NaN
16,Big Island Hawaii,Hawaii3,Hawaii III,1580.0,1778.0,n,General variables,succeeding (quasi)polity,Kingdom of Hawaii - Kamehameha Period,NaN,NaN,NaN
17,Cahokia,USMisEW,Cahokia - Early Woodland,-600.0,-150.0,n,General variables,succeeding (quasi)polity,Cahokia - Middle Woodland,NaN,NaN,NaN
19,Cahokia,USMisMW,Cahokia - Middle Woodland,-150.0,300.0,n,General variables,succeeding (quasi)polity,Cahokia - Late Woodland I,NaN,NaN,NaN


In [24]:
Relationships.head()

,NGA,PolID,Longform.Name,Polity.Start,Polity.End,Dupl,Section,Variable,Value.From,Value.To,Value.Date.From,Value.Date.To
15,Big Island Hawaii,Hawaii2,Hawaii II,1200.0,1580.0,n,General variables,relationship to preceding (quasi)polity,continuity,NaN,NaN,NaN
18,Cahokia,USMisEW,Cahokia - Early Woodland,-600.0,-150.0,n,General variables,relationship to preceding (quasi)polity,continuity,NaN,NaN,NaN
20,Cahokia,USMisMW,Cahokia - Middle Woodland,-150.0,300.0,n,General variables,relationship to preceding (quasi)polity,Continuity,NaN,NaN,NaN
22,Cahokia,USMisRo,Cahokia - Late Woodland I,300.0,450.0,n,General variables,relationship to preceding (quasi)polity,Continuity,NaN,NaN,NaN
23,Cahokia,USMisMu,Cahokia - Late Woodland II,450.0,600.0,n,General variables,relationship to preceding (quasi)polity,Continuity,NaN,NaN,NaN


In [8]:
for val in list(set(Relationships["Value.From"])):
    print(val, sum(Relationships["Value.From"]==val) )
                

cultural assimilation 25
contination 1
continuity 228
Continuity 9
not applicable 2
population migration 13
replacement 1
suspected unknown 2
indigenous revolt 1
continuation 7
elite migration 40
vassalage 1
continunity 4
Continuation 1


In [9]:
Relationships["Value.From"].replace("Continuation", "continuity", inplace = True)
Relationships["Value.From"].replace("continuation", "continuity", inplace = True)
Relationships["Value.From"].replace("continunity", "continuity", inplace = True)
Relationships["Value.From"].replace("contination", "continuity", inplace = True)
Relationships["Value.From"].replace("Continuity", "continuity", inplace = True)

                
for val in list(set(Relationships["Value.From"])):
    if val != "continuity":
        Relationships["Value.From"].replace(val, "non-continuity", inplace = True)

for val in list(set(Relationships["Value.From"])):
    print(val, sum(Relationships["Value.From"]==val) )    

non-continuity 85
continuity 250


C:\Users\Tobias\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [10]:
PolRel.loc[PolRel["Dupl"] == "y"].iloc[51:100]

,NGA,PolID,Longform.Name,Polity.Start,Polity.End,Dupl,Section,Variable,Value.From,Value.To,Value.Date.From,Value.Date.To
546,Paris Basin,ItRomPr,Roman Empire - Principate,-31.0,284.0,y,General variables,succeeding (quasi)polity,Roman Empire - Dominate,NaN,NaN,NaN
547,Paris Basin,ItRomPr,Roman Empire - Principate,-31.0,284.0,y,General variables,succeeding (quasi)polity,Roman Empire - Dominate,NaN,NaN,NaN
548,Paris Basin,ItRomPr,Roman Empire - Principate,-31.0,284.0,y,General variables,succeeding (quasi)polity,Roman Empire - Dominate,NaN,NaN,NaN
549,Paris Basin,ItRomPr,Roman Empire - Principate,-31.0,284.0,y,General variables,succeeding (quasi)polity,Roman Empire - Dominate,NaN,NaN,NaN
550,Paris Basin,TrRomDm,Roman Empire - Dominate,285.0,394.0,y,General variables,succeeding (quasi)polity,Western Roman Empire - Late Antiquity,NaN,NaN,NaN
551,Paris Basin,TrRomDm,Roman Empire - Dominate,285.0,394.0,y,General variables,relationship to preceding (quasi)polity,continuity,NaN,NaN,NaN
581,Sogdiana,IrAchae,Achaemenid Empire,-550.0,-331.0,y,General variables,succeeding (quasi)polity,Macedonian Empire,NaN,NaN,NaN
582,Sogdiana,IrAchae,Achaemenid Empire,-550.0,-331.0,y,General variables,relationship to preceding (quasi)polity,continuity,NaN,NaN,NaN
583,Sogdiana,IrSeleu,Seleucid Empire,-312.0,-63.0,y,General variables,relationship to preceding (quasi)polity,continuity,NaN,NaN,NaN
584,Sogdiana,IrSeleu,Seleucid Empire,-312.0,-63.0,y,General variables,succeeding (quasi)polity,Parthian Empire I,NaN,NaN,NaN


In [11]:
for n in list(set(PolRel["NGA"])):
    print(PolRel.loc[PolRel["NGA"]==n])

        NGA    PolID                          Longform.Name  Polity.Start  \
361  Latium  ItLatCA                    Latium - Copper Age       -3600.0   
362  Latium  ItLatCA                    Latium - Copper Age       -3600.0   
363  Latium  ItLatBA                    Latium - Bronze Age       -1800.0   
364  Latium  ItLatBA                    Latium - Bronze Age       -1800.0   
365  Latium  ItLatIA                      Latium - Iron Age       -1000.0   
366  Latium  ItLatIA                      Latium - Iron Age       -1000.0   
367  Latium  ItRomRg                          Roman Kingdom        -716.0   
368  Latium  ItRomRg                          Roman Kingdom        -716.0   
369  Latium  ItRomER                         Roman Republic        -509.0   
370  Latium  ItRomER                         Roman Republic        -509.0   
371  Latium  ItRomMR                  Middle Roman Republic        -264.0   
372  Latium  ItRomMR                  Middle Roman Republic        -264.0   

394            NaN  
              NGA    PolID       Longform.Name  Polity.Start  Polity.End Dupl  \
77  Chuuk Islands  FmTrukE  Chuuk - Early Truk        1775.0      1886.0    n   
78  Chuuk Islands  FmTrukE  Chuuk - Early Truk        1775.0      1886.0    n   
79  Chuuk Islands  FmTrukL   Chuuk - Late Truk        1886.0      1948.0    n   
80  Chuuk Islands  FmTrukL   Chuuk - Late Truk        1886.0      1948.0    n   
81  Chuuk Islands  FmTrukL   Chuuk - Late Truk        1886.0      1948.0    n   

              Section                                 Variable  \
77  General variables  relationship to preceding (quasi)polity   
78  General variables                 succeeding (quasi)polity   
79  General variables                 succeeding (quasi)polity   
80  General variables  relationship to preceding (quasi)polity   
81  General variables                 succeeding (quasi)polity   

                        Value.From  Value.To  Value.Date.From  Value.Date.To  
77              

514       NaN              NaN            NaN  
                    NGA    PolID                    Longform.Name  \
457  Niger Inland Delta  MlJeJe1                     Jenne-jeno I   
458  Niger Inland Delta  MlJeJe1                     Jenne-jeno I   
459  Niger Inland Delta  MlJeJe2                    Jenne-jeno II   
460  Niger Inland Delta  MlJeJe2                    Jenne-jeno II   
461  Niger Inland Delta  MlJeJe3                   Jenne-jeno III   
462  Niger Inland Delta  MlJeJe3                   Jenne-jeno III   
463  Niger Inland Delta  MrWagdM             Middle Wagadu Empire   
464  Niger Inland Delta  MrWagdM             Middle Wagadu Empire   
465  Niger Inland Delta  MlJeJe4                    Jenne-jeno IV   
466  Niger Inland Delta  MlJeJe4                    Jenne-jeno IV   
467  Niger Inland Delta  MrWagdL              Later Wagadu Empire   
468  Niger Inland Delta  MrWagdL              Later Wagadu Empire   
469  Niger Inland Delta  MlMali*                      M

279  Japan - Meiji Restoration       NaN              NaN            NaN  
                  NGA    PolID            Longform.Name  Polity.Start  \
835  Valley of Oaxaca  MxTieLa  Oaxaca - Tierras Largas       -1400.0   
836  Valley of Oaxaca  MxTieLa  Oaxaca - Tierras Largas       -1400.0   
837  Valley of Oaxaca  MxSanGu        Oaxaca - San Jose       -1150.0   
838  Valley of Oaxaca  MxSanGu        Oaxaca - San Jose       -1150.0   
839  Valley of Oaxaca  MxRosar         Oaxaca - Rosario        -700.0   
840  Valley of Oaxaca  MxRosar         Oaxaca - Rosario        -700.0   
841  Valley of Oaxaca  MxAlb1E      Early Monte Alban I        -500.0   
842  Valley of Oaxaca  MxAlb1E      Early Monte Alban I        -500.0   
843  Valley of Oaxaca  MxAlb1L       Monte Alban Late I        -300.0   
844  Valley of Oaxaca  MxAlb1L       Monte Alban Late I        -300.0   
845  Valley of Oaxaca  MxAlb2*           Monte Alban II        -100.0   
846  Valley of Oaxaca  MxAlb2*           Monte Al

             NGA    PolID                     Longform.Name  Polity.Start  \
286  Konya Plain  TrNeoER     Konya Plain - Early Neolithic       -9600.0   
287  Konya Plain  TrNeoCR   Konya Plain - Ceramic Neolithic       -7000.0   
288  Konya Plain  TrNeoLT      Konya Plain - Late Neolithic       -6600.0   
289  Konya Plain  TrClcER  Konya Plain - Early Chalcolithic       -6000.0   
290  Konya Plain  TrClcER  Konya Plain - Early Chalcolithic       -6000.0   
..           ...      ...                               ...           ...   
356  Konya Plain  TrOttm4                Ottoman Empire III        1683.0   
357  Konya Plain  TrOttm4                Ottoman Empire III        1683.0   
358  Konya Plain  TrOttm4                Ottoman Empire III        1683.0   
359  Konya Plain  TrOttm4                Ottoman Empire III        1683.0   
360  Konya Plain  TrOttm5                 Ottoman Empire IV        1839.0   

     Polity.End Dupl            Section  \
286     -7000.0    n  General va

         NGA    PolID           Longform.Name  Polity.Start  Polity.End Dupl  \
207  Iceland  IsCommw  Icelandic Commonwealth         930.0      1262.0    n   
208  Iceland  IsCommw  Icelandic Commonwealth         930.0      1262.0    n   
209  Iceland  NorKing       Kingdom of Norway        1262.0      1380.0    n   
210  Iceland  NorKing       Kingdom of Norway        1262.0      1380.0    n   

               Section                                 Variable  \
207  General variables  relationship to preceding (quasi)polity   
208  General variables                 succeeding (quasi)polity   
209  General variables  relationship to preceding (quasi)polity   
210  General variables                 succeeding (quasi)polity   

            Value.From  Value.To  Value.Date.From  Value.Date.To  
207         continuity       NaN              NaN            NaN  
208  Kingdom of Norway       NaN              NaN            NaN  
209         continuity       NaN              NaN            N

In [26]:
CleanRel = pd.concat((Previous,Relationships), sort=False).sort_index()

In [29]:
CleanRel.head()

,NGA,PolID,Longform.Name,Polity.Start,Polity.End,Dupl,Section,Variable,Value.From,Value.To,Value.Date.From,Value.Date.To
13,Big Island Hawaii,Hawaii1,Hawaii I,1000.0,1200.0,n,General variables,succeeding (quasi)polity,Hawaii II,NaN,NaN,NaN
14,Big Island Hawaii,Hawaii2,Hawaii II,1200.0,1580.0,n,General variables,succeeding (quasi)polity,Hawaii III,NaN,NaN,NaN
15,Big Island Hawaii,Hawaii2,Hawaii II,1200.0,1580.0,n,General variables,relationship to preceding (quasi)polity,continuity,NaN,NaN,NaN
16,Big Island Hawaii,Hawaii3,Hawaii III,1580.0,1778.0,n,General variables,succeeding (quasi)polity,Kingdom of Hawaii - Kamehameha Period,NaN,NaN,NaN
17,Cahokia,USMisEW,Cahokia - Early Woodland,-600.0,-150.0,n,General variables,succeeding (quasi)polity,Cahokia - Middle Woodland,NaN,NaN,NaN


In [37]:
TimeZero = pd.read_csv("TimeZero.csv")
TimeZero.head()

,Unnamed: 0,Orkhon Valley,Kachi Plain,Paris Basin,Cuzco,Latium,Cambodian Basin,Kansai,Middle Yellow River Valley,Niger Inland Delta,Konya Plain,Deccan,Susiana,Upper Egypt,Iceland,Yemeni Coastal Plain,Central Java,Ghanaian Coast,Sogdiana
0,T0,500,-2500,-200,1400,-700,300,400,-1800,1000,-2000,-300,-3700,-3100,1300,400,800,1800,-500


In [35]:
Relationships["Polity.End.Scaled"] = 0

# iterate over all rows in the DF
for i in range(Relationships.shape[0]):
    n = Relationships["NGA"].iloc[i]
    
    # for this NGA: read its Time_Zero from the TimeZero.csv
    T_zero = TimeZero[n]
    # Shift the time accordingly
    Relationships["Polity.End.Scaled"].iloc[i] = Relationships["Polity.End"].iloc[i] - T_zero
    

KeyError: 'Big Island Hawaii'

In [30]:

Relationships["Polity.End"]

15     1580.0
18     -150.0
20      300.0
22      450.0
23      600.0
        ...  
849     900.0
852    1715.0
858    1805.0
861    1839.0
862    1920.0
Name: Polity.End, Length: 335, dtype: float64